In [14]:
import requests
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_random_exponential


In [2]:
BASE_URI = "https://spoke.rbvi.ucsf.edu"

In [3]:
def get_api_resp(END_POINT, params=None):
    URI = BASE_URI + END_POINT
    if params:
        return requests.get(URI, params=params)
    else:
        return requests.get(URI)

In [78]:
@retry(wait=wait_random_exponential(min=10, max=30), stop=stop_after_attempt(5))
def get_context_using_api(node_value):
    type_end_point = "/api/v1/types"
    result = get_api_resp(type_end_point)
    data_spoke_types = result.json()
    node_types = list(data_spoke_types["nodes"].keys())
    edge_types = list(data_spoke_types["edges"].keys())
    node_types_to_remove = ["DatabaseTimestamp", "Version"]
    filtered_node_types = [node_type for node_type in node_types if node_type not in node_types_to_remove]
    api_params = {
        'node_filters' : filtered_node_types,
        'edge_filters': edge_types,
        'cutoff_Compound_max_phase': 3,
        'cutoff_Protein_source': ['SwissProt'],
        'cutoff_DaG_diseases_sources': ['knowledge', 'experiments'],
        'cutoff_DaG_textmining': 3,
        'cutoff_CtD_phase': 3,
        'cutoff_PiP_confidence': 0.7,
        'cutoff_ACTeG_level': ['Low', 'Medium', 'High']
    }
    node_type = "Disease"
    attribute = "name"
    nbr_end_point = "/api/v1/neighborhood/{}/{}/{}".format(node_type, attribute, node_value)
    result = get_api_resp(nbr_end_point, params=api_params)
    node_context = result.json()
    nbr_nodes = []
    nbr_edges = []
    for item in node_context:
        if "_" not in item["data"]["neo4j_type"]:
            try:
                if item["data"]["neo4j_type"] == "Protein":
                    nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["description"]))
                else:
                    nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["name"]))
            except:
                nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["identifier"]))
        elif "_" in item["data"]["neo4j_type"]:
            try:
                provenance = ", ".join(item["data"]["properties"]["sources"])
            except:
                try:
                    provenance = item["data"]["properties"]["source"]
                    if isinstance(provenance, list):
                        provenance = ", ".join(provenance)                    
                except:
                    provenance = "SPOKE-KG"                                    
            nbr_edges.append((item["data"]["source"], item["data"]["neo4j_type"], item["data"]["target"], provenance))
    nbr_nodes_df = pd.DataFrame(nbr_nodes, columns=["node_type", "node_id", "node_name"])
    nbr_edges_df = pd.DataFrame(nbr_edges, columns=["source", "edge_type", "target", "provenance"])
    merge_1 = pd.merge(nbr_edges_df, nbr_nodes_df, left_on="source", right_on="node_id").drop("node_id", axis=1)
    merge_1.loc[:,"node_name"] = merge_1.node_type + " " + merge_1.node_name
    merge_1.drop(["source", "node_type"], axis=1, inplace=True)
    merge_1 = merge_1.rename(columns={"node_name":"source"})
    merge_2 = pd.merge(merge_1, nbr_nodes_df, left_on="target", right_on="node_id").drop("node_id", axis=1)
    merge_2.loc[:,"node_name"] = merge_2.node_type + " " + merge_2.node_name
    merge_2.drop(["target", "node_type"], axis=1, inplace=True)
    merge_2 = merge_2.rename(columns={"node_name":"target"})
    merge_2 = merge_2[["source", "edge_type", "target", "provenance"]]
    merge_2.loc[:, "predicate"] = merge_2.edge_type.apply(lambda x:x.split("_")[0])
    merge_2.loc[:, "context"] =  merge_2.source + " " + merge_2.predicate.str.lower() + " " + merge_2.target + " and Provenance of this association is from " + merge_2.provenance + "."
    context = merge_2['context'].str.cat(sep=' ')
    return context



In [79]:
%%time
context = get_context_using_api("multiple sclerosis")

CPU times: user 92.2 ms, sys: 8.79 ms, total: 101 ms
Wall time: 368 ms


In [84]:
nbr_end_point = "/api/v1/neighborhood/{}/{}/{}".format(node_type, attribute, "COVID-19")
result = get_api_resp(nbr_end_point, params=api_params)
node_context = result.json()

In [91]:
nbr_nodes = []
nbr_edges = []
for item in node_context:
    if "_" not in item["data"]["neo4j_type"]:
        try:
            if item["data"]["neo4j_type"] == "Protein":
                nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["description"]))
            else:
                nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["name"]))
        except:
            nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["identifier"]))
    elif "_" in item["data"]["neo4j_type"]:
        try:
            provenance = ", ".join(item["data"]["properties"]["sources"])
        except:
            try:
                provenance = item["data"]["properties"]["source"]
                if isinstance(provenance, list):
                    provenance = ", ".join(provenance)                    
            except:
                provenance = None   
            nbr_edges.append((item["data"]["source"], item["data"]["neo4j_type"], item["data"]["target"], provenance))
    nbr_nodes_df = pd.DataFrame(nbr_nodes, columns=["node_type", "node_id", "node_name"])
    nbr_edges_df = pd.DataFrame(nbr_edges, columns=["source", "edge_type", "target", "provenance"])
    merge_1 = pd.merge(nbr_edges_df, nbr_nodes_df, left_on="source", right_on="node_id").drop("node_id", axis=1)
    merge_1.loc[:,"node_name"] = merge_1.node_type + " " + merge_1.node_name
    merge_1.drop(["source", "node_type"], axis=1, inplace=True)
    merge_1 = merge_1.rename(columns={"node_name":"source"})
    merge_2 = pd.merge(merge_1, nbr_nodes_df, left_on="target", right_on="node_id").drop("node_id", axis=1)
    merge_2.loc[:,"node_name"] = merge_2.node_type + " " + merge_2.node_name
    merge_2.drop(["target", "node_type"], axis=1, inplace=True)
    merge_2 = merge_2.rename(columns={"node_name":"target"})
    merge_2 = merge_2[["source", "edge_type", "target", "provenance"]]
    merge_2.loc[:, "predicate"] = merge_2.edge_type.apply(lambda x:x.split("_")[0])
    merge_2.loc[:, "context"] =  merge_2.source + " " + merge_2.predicate.str.lower() + " " + merge_2.target + " and Provenance of this association is from " + merge_2.provenance + "."
    context = merge_2['context'].str.cat(sep=' ')


In [98]:
merge_2.loc[:, "context"] =  merge_2.source + " " + merge_2.predicate.str.lower() + " " + merge_2.target + " and Provenance of this association is from "
merge_2[merge_2.edge_type == "INCREASEDIN_PiD"]

,source,edge_type,target,provenance,predicate,context
4,Protein Interleukin-23 subunit alpha (IL-23 su...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Interleukin-23 subunit alpha (IL-23 su...
5,Protein C-C motif chemokine 7 (Monocyte chemoa...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein C-C motif chemokine 7 (Monocyte chemoa...
6,Protein Fibroblast growth factor 2 (FGF-2) (Ba...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Fibroblast growth factor 2 (FGF-2) (Ba...
7,Protein Interleukin-13 (IL-13),INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Interleukin-13 (IL-13) increasedin Dis...
8,Protein Interleukin-2 receptor subunit alpha (...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Interleukin-2 receptor subunit alpha (...
9,Protein Interleukin-9 (IL-9) (Cytokine P40) (T...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Interleukin-9 (IL-9) (Cytokine P40) (T...
10,Protein Interleukin-7 (IL-7),INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Interleukin-7 (IL-7) increasedin Disea...
11,Protein Tumor necrosis factor ligand superfami...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein Tumor necrosis factor ligand superfami...
12,Protein C-C motif chemokine 27 (CC chemokine I...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein C-C motif chemokine 27 (CC chemokine I...
13,Protein C-C motif chemokine 3 (G0/G1 switch re...,INCREASEDIN_PiD,Disease COVID-19,None,INCREASEDIN,Protein C-C motif chemokine 3 (G0/G1 switch re...


In [89]:
context

'Disease critical COVID-19 isa Disease COVID-19 and Provenance of this association is from Disease Ontology. Disease severe COVID-19 isa Disease COVID-19 and Provenance of this association is from Disease Ontology. Disease non-severe COVID-19 isa Disease COVID-19 and Provenance of this association is from Disease Ontology. Disease COVID-19 isa Disease Coronavirus infectious disease and Provenance of this association is from Disease Ontology. Disease COVID-19 resembles Disease mucormycosis and Provenance of this association is from NCBI PubMed. Disease COVID-19 resembles Disease systemic lupus erythematosus and Provenance of this association is from NCBI PubMed. Disease COVID-19 resembles Disease cardiovascular system disease and Provenance of this association is from NCBI PubMed. Disease COVID-19 resembles Disease diabetes mellitus and Provenance of this association is from NCBI PubMed. Disease COVID-19 resembles Disease acute kidney failure and Provenance of this association is from N